In [1]:

#Import Libraries
import pickle
import math
import pandas as pd
import numpy as np
from numpy import array
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from pickle import load
from tensordash.tensordash import Tensordash


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.compat.v1.InteractiveSession(config=config)

In [3]:
dataset = pd.read_csv('All-seasons.csv/data.csv')
characters = ['Stan','Kyle','Cartman']
dataset = dataset[dataset['Character'].isin(characters)]["Line"]
dataset.shape

(24553,)

In [4]:
max_words = 50000 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dataset.values)
sequences = tokenizer.texts_to_sequences(dataset.values)
text = [item for sublist in sequences for item in sublist]
vocab_size = len(tokenizer.word_index)
print('Vocabulary size in this corpus: ', vocab_size)

Vocabulary size in this corpus:  12364


In [5]:
sentence_len = 20
pred_len = 1
train_len = sentence_len - pred_len
seq = []
# Sliding window to generate train data
for i in range(len(text)-sentence_len):
    seq.append(text[i:i+sentence_len])
# Reverse dictionary to decode tokenized sequences back to words
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
trainX = []
trainy = []
for i in seq:
    trainX.append(i[:train_len])
    trainy.append(i[-1])

In [15]:
model = Sequential([
    Embedding(vocab_size+1, 50, input_length=train_len),
    LSTM(256, return_sequences=True),
    LSTM(256),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

ValueError: Input 0 of layer lstm_8 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 128]

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 19, 50)            618250    
_________________________________________________________________
lstm_4 (LSTM)                (None, 19, 150)           120600    
_________________________________________________________________
lstm_5 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_8 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_9 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_10 (Dense)             (None, 150)               22650     
_________________________________________________________________
dense_11 (Dense)             (None, 12364)            

In [8]:
histories = Tensordash(ModelName = 'South Park Dialogue',
                       email = 'pranjal.kalbag@gmail.com')

In [9]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
with tf.device("/gpu:0"):
    model.fit(np.asarray(trainX), pd.get_dummies(np.asarray(trainy)), batch_size=128, epochs=30, callbacks = [histories])

Epoch 1/30
2068/2068 [==============================] - 49s 24ms/step - loss: 6.4658 - accuracy: 0.0376
Epoch 2/30
2068/2068 [==============================] - 82s 39ms/step - loss: 5.9775 - accuracy: 0.0644
Epoch 3/30
2068/2068 [==============================] - 92s 44ms/step - loss: 5.7035 - accuracy: 0.0865
Epoch 4/30
2068/2068 [==============================] - 93s 45ms/step - loss: 5.5667 - accuracy: 0.0941
Epoch 5/30
2068/2068 [==============================] - 101s 49ms/step - loss: 5.3919 - accuracy: 0.1057
Epoch 6/30
2068/2068 [==============================] - 92s 45ms/step - loss: 5.2714 - accuracy: 0.1137
Epoch 7/30
2068/2068 [==============================] - 94s 46ms/step - loss: 5.1585 - accuracy: 0.1204
Epoch 8/30
2068/2068 [==============================] - 96s 47ms/step - loss: 5.0633 - accuracy: 0.1256
Epoch 9/30
2068/2068 [==============================] - 97s 47ms/step - loss: 4.9828 - accuracy: 0.1302
Epoch 10/30
2068/2068 [==============================] - 99s 48

In [10]:
input_string = input("Enter String:")
max_len = 20
tokenized_sent = tokenizer.texts_to_sequences([input_string])
max_len = max_len+len(tokenized_sent[0])
while len(tokenized_sent[0]) < max_len:
        padded_sentence = pad_sequences(tokenized_sent[-19:],maxlen=19)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent[0].append(op.argmax()+1)
pred_string = " ".join(map(lambda x : reverse_word_map[x],tokenized_sent[0]))
print(pred_string)

why should i do it to get it to the time and i don't know what i don't know what i waunt i
